# Create Master files
- generic features - master_clean_df.csv
- adverse events table - master_ae.csv
- dropout reason table - master_clean_dropout.csv

## Master clean table

In [ ]:
# Import necessary packages:

import pandas as pd
import numpy as np
from collections import Counter

import matplotlib
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

import seaborn as sns

import scipy.stats as spstats
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#download cleaned data 
depression_df = pd.read_csv("/content/drive/MyDrive/HIDS510/Collab Notebooks/Depression Notebooks/depression_clean_df.csv")
depression_df['disease_type'] = "Depression"
anxiety_df = pd.read_csv("/content/drive/MyDrive/HIDS510/Collab Notebooks/Anxiety Notebooks/anxiety_clean_df.csv")
anxiety_df['disease_type'] = "Anxiety"
bipolar_df = pd.read_csv("/content/drive/MyDrive/HIDS510/Collab Notebooks/Bipolar Notebooks/bipolar_clean_df.csv")
bipolar_df['disease_type'] = "Bipolar"
alzheimers_df = pd.read_csv("/content/drive/MyDrive/HIDS510/Collab Notebooks/Alzheimer's Disease Notebook/alzheimers_clean_df.csv")
alzheimers_df['disease_type'] = "Alzheimer's Disease"
parkinsons_df = pd.read_csv("/content/drive/MyDrive/HIDS510/Collab Notebooks/Parkinson's Disease Notebooks/parkinsons_clean_df.csv")
parkinsons_df['disease_type'] = "Parkinson's Disease"
als_df = pd.read_csv("/content/drive/MyDrive/HIDS510/Collab Notebooks/Amyotrophic Lateral Sclerosis (ALS) Notebooks/als_clean_df.csv")
als_df['disease_type'] = "Amyotrophic lateral sclerosis"

In [ ]:
# concat all clean dfs tp get master_df
frames = [depression_df, anxiety_df, bipolar_df, alzheimers_df, parkinsons_df, als_df]
master = pd.concat(frames)

#drop first column of no data 
master_df = master.iloc[: , 1:]
master_df.to_csv("/content/drive/MyDrive/HIDS510/Collab Notebooks/Master Notebooks/master_clean_df.csv")

## Master AE Event Table

In [ ]:
#download file with ae events from ALL diseases adverseEventTable.ipynb
depression_ae_df = pd.read_csv("/content/drive/MyDrive/HIDS510/Collab Notebooks/Adverse Events Table/Final AE Tables/depression_ae_table.csv")
anxiety_ae_df = pd.read_csv("/content/drive/MyDrive/HIDS510/Collab Notebooks/Adverse Events Table/Final AE Tables/anxiety_ae_table.csv")
bipolar_ae_df = pd.read_csv("/content/drive/MyDrive/HIDS510/Collab Notebooks/Adverse Events Table/Final AE Tables/bipolar_ae_table.csv")
alzheimers_ae_df = pd.read_csv("/content/drive/MyDrive/HIDS510/Collab Notebooks/Adverse Events Table/Final AE Tables/alzheimers_ae_table.csv")
parkinsons_ae_df = pd.read_csv("/content/drive/MyDrive/HIDS510/Collab Notebooks/Adverse Events Table/Final AE Tables/parkinsons_ae_table.csv")
als_ae_df = pd.read_csv("/content/drive/MyDrive/HIDS510/Collab Notebooks/Adverse Events Table/Final AE Tables/als_ae_table.csv")

#download anxiety_ae_key
depression_ae_key = pd.read_csv("/content/drive/MyDrive/HIDS510/Collab Notebooks/Adverse Events Table/subset_adverse_events files/depression_ae_key.csv", index_col=0)
anxiety_ae_key = pd.read_csv("/content/drive/MyDrive/HIDS510/Collab Notebooks/Adverse Events Table/subset_adverse_events files/anxiety_ae_key.csv", index_col=0)
bipolar_ae_key = pd.read_csv("/content/drive/MyDrive/HIDS510/Collab Notebooks/Adverse Events Table/subset_adverse_events files/bipolar_ae_key.csv", index_col=0)
alzheimers_ae_key = pd.read_csv("/content/drive/MyDrive/HIDS510/Collab Notebooks/Adverse Events Table/subset_adverse_events files/alzheimers_ae_key.csv", index_col=0)
parkinsons_ae_key = pd.read_csv("/content/drive/MyDrive/HIDS510/Collab Notebooks/Adverse Events Table/subset_adverse_events files/parkinsons_ae_key.csv", index_col=0)
als_ae_key = pd.read_csv("/content/drive/MyDrive/HIDS510/Collab Notebooks/Adverse Events Table/subset_adverse_events files/als_ae_key.csv", index_col=0)

In [ ]:
# concat all ae dfs to get master_df
frames = [depression_ae_df, anxiety_ae_df, bipolar_ae_df, alzheimers_ae_df, parkinsons_ae_df, als_ae_df]
master_ae_df = pd.concat(frames)

In [ ]:
# concat all ae keys to get master_key
frames = [depression_ae_key, anxiety_ae_key, bipolar_ae_key, alzheimers_ae_key, parkinsons_ae_key, als_ae_key]
master_ae_key = pd.concat(frames)

In [ ]:
#merge dataframes (master_ae_df and master_ae_key)
merged_master_df = master_ae_df.merge(master_ae_key, on='id', how='left')

In [ ]:
#get subject count by trial into the its respective ae

#drop id column 
ae_drop_id = merged_master_df.drop(['id'], axis=1)

ae_drop_id

,Influenza,Tic,Rash,Pain,Mania,Crime,Anger,Wound,Dizziness,Muscle spasms,...,Central venous catheterisation,Gastrointestinal tube insertion,Clostridium difficile infection,Adjustment disorder with anxiety,Pulmonary function test decreased,Gastrointestinal fungal infection,Type IV hypersensitivity reaction,Respiratory syncytial virus bronchitis,nct_id,subjects_affected
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NCT03315208,1.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NCT03315208,1.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NCT03315208,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NCT03315208,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NCT03315208,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39956,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NCT03160898,6.0
39957,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NCT03160898,2.0
39958,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NCT03160898,1.0
39959,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NCT03160898,2.0


In [ ]:
#get list of ae column names
ae_column_df = ae_drop_id.iloc[:, 0:1953]

#get list of ae columns 
ae_list = list(ae_column_df.columns)

ae_drop_id_list = list(ae_drop_id.columns)

In [ ]:
#sub all values of NAN in the ae columns with 0 
ae_drop_id[ae_list] = ae_drop_id[ae_list].T.fillna(0).T

In [ ]:
# fill all flag values == 1 with nan
ae_drop_id[ae_list] = ae_drop_id[ae_list].replace(1, np.NaN)

# replace nan values in the 479 ae type columns with the corresponding count value from subjects_affected column
ae_drop_id[ae_list] = ae_drop_id[ae_list].T.fillna(ae_drop_id['subjects_affected']).T

#group responses from the merged column (type of event and count) by nct_id
ae_final_df = ae_drop_id.groupby('nct_id').sum()

#drop column subjects_affected_total 
ae_final_df = ae_final_df.drop(['subjects_affected'], axis=1)

#check this ae table 
#ae_final_df

In [ ]:
#merge to get V3_master_df
#V3_master_df = ae_final_df.merge(master_df, on='nct_id', how='left')

In [ ]:
#V3_master_df

Check for any ae columns that have only zeros and drop

In [ ]:
ae_final_df.loc[:, (ae_final_df != 0).any(axis=0)]

,Influenza,Tic,Rash,Pain,Mania,Crime,Anger,Wound,Dizziness,Muscle spasms,...,Metastatic bronchial carcinoma,Electrocardiogram QT prolonged,Central venous catheterisation,Gastrointestinal tube insertion,Clostridium difficile infection,Adjustment disorder with anxiety,Pulmonary function test decreased,Gastrointestinal fungal infection,Type IV hypersensitivity reaction,Respiratory syncytial virus bronchitis
nct_id,,,,,,,,,,,,,,,,,,,,,
NCT00010803,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NCT00035815,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NCT00037830,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NCT00070941,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,14.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NCT00071110,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NCT04322526,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NCT04403399,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NCT04468347,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
ae_final_df.to_csv("/content/drive/MyDrive/HIDS510/Collab Notebooks/Master Notebooks/master_ae.csv")

## Dropout Reason Table

In [ ]:
# clean nct_ids 
depression_nct_ids = depression_df['nct_id']
anxiety_nct_ids = anxiety_df['nct_id']
bipolar_nct_ids = bipolar_df['nct_id']
alzheimers_nct_ids = alzheimers_df['nct_id']
parkinsons_nct_ids = parkinsons_df['nct_id']
als_nct_ids = als_df['nct_id']

In [ ]:
#download cleaned master data 
pd.set_option('display.max_columns', None)
depression_dropout_df= pd.read_csv('/content/drive/MyDrive/HIDS510/Collab Notebooks/NLP Dropout Reason/Dropout Output Files/depression_dropout_df.csv', index_col=0)
depression_dropout_df.rename(columns = {'label':'dropout_reason'}, inplace = True)
anxiety_dropout_df= pd.read_csv('/content/drive/MyDrive/HIDS510/Collab Notebooks/NLP Dropout Reason/Dropout Output Files/anxiety_dropout_df.csv', index_col=0)
anxiety_dropout_df.rename(columns = {'label':'dropout_reason'}, inplace = True)
bipolar_dropout_df= pd.read_csv('/content/drive/MyDrive/HIDS510/Collab Notebooks/NLP Dropout Reason/Dropout Output Files/bipolar_dropout_df.csv', index_col=0)
bipolar_dropout_df.rename(columns = {'label':'dropout_reason'}, inplace = True)
alzheimers_dropout_df= pd.read_csv('/content/drive/MyDrive/HIDS510/Collab Notebooks/NLP Dropout Reason/Dropout Output Files/alzheimers_dropout_df.csv', index_col=0)
alzheimers_dropout_df.rename(columns = {'label':'dropout_reason'}, inplace = True)
parkinsons_dropout_df= pd.read_csv('/content/drive/MyDrive/HIDS510/Collab Notebooks/NLP Dropout Reason/Dropout Output Files/parkinsons_dropout_df.csv', index_col=0)
parkinsons_dropout_df.rename(columns = {'label':'dropout_reason'}, inplace = True)
als_dropout_df= pd.read_csv('/content/drive/MyDrive/HIDS510/Collab Notebooks/NLP Dropout Reason/Dropout Output Files/als_dropout_df.csv', index_col=0)
als_dropout_df.rename(columns = {'label':'dropout_reason'}, inplace = True)

### Clean master dropout reason 

In [ ]:
#get subject count by trial into the its respective dropout reason

#drop id column 
depression_dropout_drop_id = depression_dropout_df.drop(['id', 'reason', 'disease_type', 'disease_group'], axis = 1)
anxiety_dropout_drop_id = anxiety_dropout_df.drop(['id', 'reason', 'disease_type', 'disease_group'], axis = 1)
bipolar_dropout_drop_id = bipolar_dropout_df.drop(['id', 'reason', 'disease_type', 'disease_group'], axis = 1)
alzheimers_dropout_drop_id = alzheimers_dropout_df.drop(['id', 'reason', 'disease_type', 'disease_group'], axis = 1)
parkinsons_dropout_drop_id = parkinsons_dropout_df.drop(['id', 'reason', 'disease_type', 'disease_group'], axis = 1)
als_dropout_drop_id = als_dropout_df.drop(['id', 'reason', 'disease_type', 'disease_group'], axis = 1)

In [ ]:
depression_dropout_drop_id = pd.merge(depression_dropout_drop_id, depression_nct_ids, on='nct_id', how='inner')
anxiety_dropout_drop_id = pd.merge(anxiety_dropout_drop_id, anxiety_nct_ids, on='nct_id', how='inner')
bipolar_dropout_drop_id = pd.merge(bipolar_dropout_drop_id, bipolar_nct_ids, on='nct_id', how='inner')
alzheimers_dropout_drop_id = pd.merge(alzheimers_dropout_drop_id, alzheimers_nct_ids, on='nct_id', how='inner')
parkinsons_dropout_drop_id = pd.merge(parkinsons_dropout_drop_id, parkinsons_nct_ids, on='nct_id', how='inner')
als_dropout_drop_id = pd.merge(als_dropout_drop_id, als_nct_ids, on='nct_id', how='inner')

In [ ]:
# One-hot-encode categorical:
prefix_cols = ['dropout_reason']
dummy_cols = ['dropout_reason']
              
depression_dropout_df = pd.get_dummies(depression_dropout_drop_id, prefix=prefix_cols, columns=dummy_cols)
anxiety_dropout_df = pd.get_dummies(anxiety_dropout_drop_id, prefix=prefix_cols, columns=dummy_cols)
bipolar_dropout_df = pd.get_dummies(bipolar_dropout_drop_id, prefix=prefix_cols, columns=dummy_cols)
alzheimers_dropout_df = pd.get_dummies(alzheimers_dropout_drop_id, prefix=prefix_cols, columns=dummy_cols)
parkinsons_dropout_df = pd.get_dummies(parkinsons_dropout_drop_id, prefix=prefix_cols, columns=dummy_cols)
als_dropout_df = pd.get_dummies(als_dropout_drop_id, prefix=prefix_cols, columns=dummy_cols)

In [ ]:
depression_dropout_df

,nct_id,count,dropout_reason_adverse event,dropout_reason_consent withdrawn,dropout_reason_death,dropout_reason_inclusion/exclusion criteria issue,dropout_reason_lack of efficacy,dropout_reason_lost to follow-up,dropout_reason_physician decision,dropout_reason_protocol violation,dropout_reason_subject withdrawal
0,NCT00000378,16.0,0,0,0,0,1,0,0,0,0
1,NCT00000378,14.0,0,0,0,0,1,0,0,0,0
2,NCT00027378,0.0,0,0,0,0,0,0,1,0,0
3,NCT00027378,3.0,0,0,0,0,0,0,1,0,0
4,NCT00043550,12.0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
6725,NCT04524598,0.0,0,0,0,1,0,0,0,0,0
6726,NCT04524598,6.0,0,0,0,1,0,0,0,0,0
6727,NCT04524598,0.0,0,0,0,0,0,0,0,0,1
6728,NCT04524598,0.0,0,0,0,0,0,1,0,0,0


In [ ]:
#get list of ae column names
depression_dropout_column_df = depression_dropout_df.iloc[:, 2:12]
anxiety_dropout_column_df = anxiety_dropout_df.iloc[:, 2:12]
bipolar_dropout_column_df = bipolar_dropout_df.iloc[:, 2:12]
alzheimers_dropout_column_df = alzheimers_dropout_df.iloc[:, 2:12]
parkinsons_dropout_column_df = parkinsons_dropout_df.iloc[:, 2:12]
als_dropout_column_df = als_dropout_df.iloc[:, 2:12]

#get list of ae columns 
depression_dropout_list = list(depression_dropout_column_df.columns)
anxiety_dropout_list = list(anxiety_dropout_column_df.columns)
bipolar_dropout_list = list(bipolar_dropout_column_df.columns)
alzheimers_dropout_list = list(alzheimers_dropout_column_df.columns)
parkinsons_dropout_list = list(parkinsons_dropout_column_df.columns)
als_dropout_list = list(als_dropout_column_df.columns)

In [ ]:
#depression
# fill all flag values == 1 with nan
depression_dropout_df[depression_dropout_list] = depression_dropout_df[depression_dropout_list].replace(1, np.NaN)

# replace nan values in the 479 ae type columns with the corresponding count value from subjects_affected column
depression_dropout_df[depression_dropout_list] = depression_dropout_df[depression_dropout_list].T.fillna(depression_dropout_df['count']).T

#group responses from the merged column (type of event and count) by nct_id
dropout_final_df = depression_dropout_df.groupby('nct_id').sum()

#drop column subjects_affected_total 
depression_dropout_final_df = dropout_final_df.drop(['count'], axis=1)

depression_dropout_final_df['disease_type'] = "Depression"

#check this ae table 
#ae_final_df
depression_dropout_final_df.to_csv("/content/drive/MyDrive/HIDS510/Collab Notebooks/NLP Dropout Reason/Dropout Output Files/depression_clean_dropout.csv")

In [ ]:
depression_dropout_final_df.shape

(483, 10)

In [ ]:
#anxiety
# fill all flag values == 1 with nan
anxiety_dropout_df[anxiety_dropout_list] = anxiety_dropout_df[anxiety_dropout_list].replace(1, np.NaN)

# replace nan values in the 479 ae type columns with the corresponding count value from subjects_affected column
anxiety_dropout_df[anxiety_dropout_list] = anxiety_dropout_df[anxiety_dropout_list].T.fillna(anxiety_dropout_df['count']).T

#group responses from the merged column (type of event and count) by nct_id
dropout_final_df = anxiety_dropout_df.groupby('nct_id').sum()

#drop column subjects_affected_total 
anxiety_dropout_final_df = dropout_final_df.drop(['count'], axis=1)

anxiety_dropout_final_df['disease_type'] = "Anxiety"

#check this ae table 
#ae_final_df
anxiety_dropout_final_df.to_csv("/content/drive/MyDrive/HIDS510/Collab Notebooks/NLP Dropout Reason/Dropout Output Files/anxiety_clean_dropout.csv")

In [ ]:
anxiety_dropout_final_df.shape

(44, 8)

In [ ]:
#bipolar
# fill all flag values == 1 with nan
bipolar_dropout_df[bipolar_dropout_list] = bipolar_dropout_df[bipolar_dropout_list].replace(1, np.NaN)

# replace nan values in the 479 ae type columns with the corresponding count value from subjects_affected column
bipolar_dropout_df[bipolar_dropout_list] = bipolar_dropout_df[bipolar_dropout_list].T.fillna(bipolar_dropout_df['count']).T

#group responses from the merged column (type of event and count) by nct_id
dropout_final_df = bipolar_dropout_df.groupby('nct_id').sum()

#drop column subjects_affected_total 
bipolar_dropout_final_df = dropout_final_df.drop(['count'], axis=1)

bipolar_dropout_final_df['disease_type'] = "Bipolar"

#check this ae table 
#ae_final_df
bipolar_dropout_final_df.to_csv("/content/drive/MyDrive/HIDS510/Collab Notebooks/NLP Dropout Reason/Dropout Output Files/bipolar_clean_dropout.csv")

In [ ]:
bipolar_dropout_final_df.shape

(190, 10)

In [ ]:
#alheimers
# fill all flag values == 1 with nan
alzheimers_dropout_df[alzheimers_dropout_list] = alzheimers_dropout_df[alzheimers_dropout_list].replace(1, np.NaN)

# replace nan values in the 479 ae type columns with the corresponding count value from subjects_affected column
alzheimers_dropout_df[alzheimers_dropout_list] = alzheimers_dropout_df[alzheimers_dropout_list].T.fillna(alzheimers_dropout_df['count']).T

#group responses from the merged column (type of event and count) by nct_id
dropout_final_df = alzheimers_dropout_df.groupby('nct_id').sum()

#drop column subjects_affected_total 
alzheimers_dropout_final_df = dropout_final_df.drop(['count'], axis=1)

alzheimers_dropout_final_df['disease_type'] = "Alzheimer's Disease"

#check this ae table 
#ae_final_df
alzheimers_dropout_final_df.to_csv("/content/drive/MyDrive/HIDS510/Collab Notebooks/NLP Dropout Reason/Dropout Output Files/alzheimers_clean_dropout.csv")

In [ ]:
alzheimers_dropout_final_df.shape

(233, 10)

In [ ]:
#parkinsons
# fill all flag values == 1 with nan
parkinsons_dropout_df[parkinsons_dropout_list] = parkinsons_dropout_df[parkinsons_dropout_list].replace(1, np.NaN)

# replace nan values in the 479 ae type columns with the corresponding count value from subjects_affected column
parkinsons_dropout_df[parkinsons_dropout_list] = parkinsons_dropout_df[parkinsons_dropout_list].T.fillna(parkinsons_dropout_df['count']).T

#group responses from the merged column (type of event and count) by nct_id
dropout_final_df = parkinsons_dropout_df.groupby('nct_id').sum()

#drop column subjects_affected_total 
parkinsons_dropout_final_df = dropout_final_df.drop(['count'], axis=1)

parkinsons_dropout_final_df['disease_type'] = "Parkinson's Disease"

#check this ae table 
#ae_final_df
parkinsons_dropout_final_df.to_csv("/content/drive/MyDrive/HIDS510/Collab Notebooks/NLP Dropout Reason/Dropout Output Files/parkinsons_clean_dropout.csv")

In [ ]:
parkinsons_dropout_final_df.shape

(249, 10)

In [ ]:
#als
# fill all flag values == 1 with nan
als_dropout_df[als_dropout_list] = als_dropout_df[als_dropout_list].replace(1, np.NaN)

# replace nan values in the 479 ae type columns with the corresponding count value from subjects_affected column
als_dropout_df[als_dropout_list] = als_dropout_df[als_dropout_list].T.fillna(als_dropout_df['count']).T

#group responses from the merged column (type of event and count) by nct_id
dropout_final_df = als_dropout_df.groupby('nct_id').sum()

#drop column subjects_affected_total 
als_dropout_final_df = dropout_final_df.drop(['count'], axis=1)

als_dropout_final_df['disease_type'] = "ALS"

#check this ae table 
#ae_final_df
als_dropout_final_df.to_csv("/content/drive/MyDrive/HIDS510/Collab Notebooks/NLP Dropout Reason/Dropout Output Files/als_clean_dropout.csv")

In [ ]:
als_dropout_final_df.shape

(1, 2)

In [ ]:
# concat all clean dfs tp get master_df
frames = [depression_dropout_final_df, anxiety_dropout_final_df, bipolar_dropout_final_df, alzheimers_dropout_final_df, parkinsons_dropout_final_df, als_dropout_final_df]
master_dropout_final_df = pd.concat(frames)

In [ ]:
master_dropout_final_df

,dropout_reason_adverse event,dropout_reason_consent withdrawn,dropout_reason_death,dropout_reason_inclusion/exclusion criteria issue,dropout_reason_lack of efficacy,dropout_reason_lost to follow-up,dropout_reason_physician decision,dropout_reason_protocol violation,dropout_reason_subject withdrawal,disease_type
nct_id,,,,,,,,,,
NCT00000378,0.0,0.0,0.0,0.0,30.0,0.0,0.0,0.0,0.0,Depression
NCT00027378,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,Depression
NCT00043550,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,54.0,Depression
NCT00056316,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,4.0,Depression
NCT00070941,22.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Depression
...,...,...,...,...,...,...,...,...,...,...
NCT03857867,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,Parkinson's Disease
NCT04082663,6.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Parkinson's Disease
NCT04164758,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Parkinson's Disease


In [ ]:
master_dropout_final_df.isna().sum().sum()

96

In [ ]:
master_dropout_final_df = master_dropout_final_df.replace(np.nan, 0)

In [ ]:
master_dropout_final_df

,dropout_reason_adverse event,dropout_reason_consent withdrawn,dropout_reason_death,dropout_reason_inclusion/exclusion criteria issue,dropout_reason_lack of efficacy,dropout_reason_lost to follow-up,dropout_reason_physician decision,dropout_reason_protocol violation,dropout_reason_subject withdrawal,disease_type
nct_id,,,,,,,,,,
NCT00000378,0.0,0.0,0.0,0.0,30.0,0.0,0.0,0.0,0.0,Depression
NCT00027378,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,Depression
NCT00043550,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,54.0,Depression
NCT00056316,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,4.0,Depression
NCT00070941,22.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Depression
...,...,...,...,...,...,...,...,...,...,...
NCT03857867,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,Parkinson's Disease
NCT04082663,6.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Parkinson's Disease
NCT04164758,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Parkinson's Disease


In [ ]:
master_dropout_final_df.to_csv("/content/drive/MyDrive/HIDS510/Collab Notebooks/NLP Dropout Reason/Dropout Output Files/master_clean_dropout.csv")